<a href="https://colab.research.google.com/github/yeaeunJi/deep_learning-/blob/main/%ED%95%99%EC%8A%B5%EA%B4%80%EB%A0%A8%EA%B8%B0%EC%88%A0%EB%93%A4(1)_%EB%A7%A4%EA%B0%9C%EB%B3%80%EC%88%98%EA%B0%B1%EC%8B%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 매개변수 갱신
- 신경망의 목적 : 손실 함수의 값을 가능한 한 낮추는 매개변수 찾기

   ==> 최적화(optimization)

- 확률적 경사 하강법(SGD) : 최적의 매개변수 값을 찾는 방법 중 하나로 매개변수의 기울기(미분)을 통해서 기울어진 방향으로 매개변수 값을 갱신을 반복함으로써 최적의 매개변수 값을 찾고자 함

- SGD의 단점과 다른 최적화 기법을 배우고자 함

### 확률적 경사 하강법(SGD)

In [1]:
class SGD :
  def __init__(self, lr = 0.01) :
    self.lr = lr
  
  def update(self, params, grads) :
    for key in params.keys() :
      params[key] -=  self.lr * grads[key] # params['W1'] : 가중치, grads['W1'] : 기울기

# SGD 클래스를 사용한 신경망 매개변수의 진행 예
# network = TwoLayerNet(매개변수)
# optimizer = SGD()

# for i in range(1000) :
#   x_batch, y_batch = get_mini_batch(...) 
#   grads = network.gradient(x_batch, t_batch)
#   params = network.params
#   optimizer.update(params, grads)

### SGD 단점
- 단순하고 구현이 쉽지만 경우에 따라 비효율적일 때가 존재함
  - 비등방성(anisotropy) 함수(방향에 따라 기울기가 달리지는 함수)에서 매개변수의 기울기 탐색 경로가 비효율적(지그재그로 이동)

### 모멘텀(Momentum)
-  '운동량'을 의미하는 단어